In [1]:
import glob, os, ultralytics, PIL, pickle, json
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm

from DatasetConversion import *

PIL.Image.MAX_IMAGE_PIXELS = 933120000

C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\DatasetConversion.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
input_folder = "D:\FloodChange\AAA_HistoricalIndexes"
output_folder = "D:\FloodChange\AAA_HistoricalIndexes_resized"
output_dataset_dir = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\BBNN\tileindices\augmentv4\preds\output_dataset\\"
model_weights = r'C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\weights\best.pt'
inference = r"C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions"

target_size = (512, 512) 
original_shapes = []

# COCO DATASET PARAMS
category_labels = {
    0 : "County",
    1 : "Tile",
    2 : "Box",
    3 : "Legend"
}

categories=[0, 1]

In [3]:
# THIS CODE RESIZES TO A CONSISTENT 512 x 512, WHICH WAS USED FOR TRAINING
img_dir = f"{output_dataset_dir}/img"

os.makedirs(img_dir, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

for filename in tqdm(os.listdir(input_folder)):

    # DEFINE DIRS
    input_path  = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)
    copy_path   = os.path.join(img_dir, filename)
    
    if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.bmp', '.gif', '.tif')):
        if not os.path.exists(output_path[:-3]+"png"):
            try:
                # OPEN IMAGE
                img = np.asarray(Image.open(input_path))
                original_shapes.append(img.shape)

                # RESCALE BW IMAGES TO UINT8 TO BE ABLE TO USE BILINEAR INTERPOLATION
                if np.max(img) != 255:
                    img = 255 * img / np.max(img)
                img = Image.fromarray(img.astype(np.uint8))
                # img.save(copy_path[:-3]+"png")

                # RESIZE AND SAVE
                img_resized = img.resize(target_size, PIL.Image.BILINEAR)
                img_resized.save(output_path[:-3]+"png")
            except: 
                continue

  0%|          | 0/40 [00:00<?, ?it/s]

In [10]:
model = ultralytics.YOLO(model_weights)

results = model(glob.glob(output_folder+"\\*"), imgsz=512, conf=0.6,)


0: 512x512 4 tiles, 1 county, 1: 512x512 22 tiles, 1 county, 2: 512x512 76 tiles, 1 county, 3: 512x512 22 tiles, 1 county, 4: 512x512 76 tiles, 1 county, 5: 512x512 39 tiles, 1 county, 6: 512x512 39 tiles, 1 county, 7: 512x512 39 tiles, 1 county, 8: 512x512 39 tiles, 1 county, 9: 512x512 5 tiles, 1 county, 10: 512x512 2 tiles, 1 county, 11: 512x512 5 tiles, 1 county, 12: 512x512 4 tiles, 1 county, 13: 512x512 118 tiles, 1 county, 14: 512x512 4 tiles, 1 county, 15: 512x512 53 tiles, 1 county, 16: 512x512 27 tiles, 1 county, 17: 512x512 16 tiles, 1 county, 18: 512x512 199 tiles, 1 county, 19: 512x512 60 tiles, 1 county, 20: 512x512 36 tiles, 1 county, 21: 512x512 5 tiles, 1 county, 22: 512x512 71 tiles, 1 county, 23: 512x512 58 tiles, 1 county, 24: 512x512 35 tiles, 1 county, 25: 512x512 41 tiles, 1 county, 26: 512x512 49 tiles, 1 county, 27: 512x512 62 tiles, 1 county, 28: 512x512 197 tiles, 1 county, 29: 512x512 117 tiles, 1 county, 30: 512x512 15 tiles, 1 county, 31: 512x512 4 tiles,

In [12]:
os.makedirs(inference, exist_ok=True)

# Process results list
for result in results:
    fn = os.path.join(inference, os.path.basename(result.path))
    print(fn)
    result.save(filename=os.path.join(inference, os.path.basename(result.path)))
    # result.plot(save=True, filename=fn)

C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480233IND0_0382.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480287IND0_0281.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480287IND0_0288.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480287IND0_0382.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480287IND0_0985.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480296IND0_0982.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480296IND0_0985.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480296IND0_0987.png
C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\YOLO_dataset\runs\detect\train8\predictions\480296IND0_1279.png
C

In [22]:
pkl_dir = f"{output_dataset_dir}/pkl"
os.makedirs(pkl_dir, exist_ok=True)


for i, result in enumerate(results[0]):

    fn = os.path.basename(result[0].path)
    
    y_shape, x_shape = original_shapes[i]
    curr_dict = {'labels' : result.boxes.cls.to('cpu').numpy().tolist(), 'boxes' : []}

    boxes = result.boxes.xyxyn.cpu().numpy()
    # print(boxes)
    boxes[:, [0, 2]] = boxes[:, [0, 2]] * x_shape
    boxes[:, [1, 3]] = boxes[:, [1, 3]] * y_shape
    # print(boxes)
    curr_dict['boxes'] = np.vstack([boxes[:, 1], boxes[:, 2], boxes[:, 3], boxes[:, 0]]).T
    # print(curr_dict['boxes'])
    # error
    
    
    with open(os.path.join(pkl_dir, fn[:-3]+"pkl"), 'wb') as handle:
        pickle.dump(curr_dict, handle)

In [23]:
annotations_dir = f"{output_dataset_dir}annotations\\"
os.makedirs(annotations_dir, exist_ok=True)

data = toCOCO(img_dir, pkl_dir, f"{annotations_dir}annotations.json", "", category_labels, categories)

  0%|          | 0/40 [00:00<?, ?it/s]

[       1889      2580.5      3001.5      1612.5]
1612.4583 3001.4807 968.02344 1112.4895
[     762.06      2582.6      1889.5      1612.3]
1612.2798 1889.4956 970.36816 1127.4382
[      763.2      1615.9      1892.8      637.63]
637.6258 1892.7802 978.30963 1129.585
[     1875.1      1626.7      3006.6      635.89]
635.8889 3006.585 990.81006 1131.4786
[       1191      503.42      1474.6      348.59]
348.58618 1474.5973 154.83435 283.63794
[     1185.3      1780.5      1459.6      1627.8]
1627.8115 1459.5854 152.7179 274.28406
[     1458.5      1764.3      1741.9      1624.4]
1624.4443 1741.8998 139.83801 283.40344
[     1464.7        1875      1743.5      1756.6]
1756.6 1743.5322 118.43262 278.85706
[       1472      661.04      1752.2      505.56]
505.5555 1752.2482 155.47983 280.2185
[     1182.8      873.79      1465.5      714.21]
714.2053 1465.4517 159.58142 282.60547
[     1183.9      1049.2      1463.8       875.7]
875.6966 1463.7683 173.53351 279.9131
[     1745.4      1760.